# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: _____

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [32]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emoji
import pandas as pd
import re
from emoji import UNICODE_EMOJI
import emoji

In [21]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [15]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [16]:
#Produto escolhido:
produto = 'Windows'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

Capturando os dados do twitter:

In [19]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q = produto, lang=lang, tweet_mode="extended").items():
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
            break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [20]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [92]:
Doc = pd.read_excel('Windows.xlsx', sheet_name = 0)
Doc.head()

,Treinamento,Relevancia
0,@evelynchartres oh that's good to know! i have...,1
1,how to resolve a usb security key is not worki...,1
2,"kumo ind. llc korea\naluminum &amp; u-pvc, win...",0
3,full time: heritage windows 2 - 1 braishfield ...,0
4,@sagaciouszu @aciouvistas @gorydetail it doesn...,1


In [94]:
limp=[]
for i in Doc.Treinamento:
    a=re.sub('@','',i)
    a=re.sub(':','',a)
    a=re.sub("'",'',a)
    a=re.sub('-','',a)
    a=re.sub('#','',a)
    a=re.sub('"','',a)
    a=re.sub('!','',a)
    a=re.sub(';','',a)
    a=re.sub('\n',' ',a)
    a=re.sub('\t',' ',a)
    a=re.sub('&',' ',a)
    a=a.strip('()')
    a=a.replace('.','')
    a=a.replace('?','')
    a=a.replace(',','')
    limp.append(a)
print(limp)

['evelynchartres oh thats good to know i have windows 6 haha i really need a new computer', 'how to resolve a usb security key is not working on windows 10  https//tco/haugaqmie0 https//tco/gzja1nk7ug', 'kumo ind llc korea aluminum  amp upvc windows doors modern kitchen trading  showroom sheikh mohammed bin salem road ras al khaimah uae from saturday  thursday 0900am100pm then 400pm1000pm  factory umm al quwain uae from saturday  thursday 700am  700pm https//tco/kplxyscls4', 'full time heritage windows 2  1 braishfield football club herbra pitchero https//tco/zl7ulrkfe8 https//tco/ffqw0sgspv', 'sagaciouszu aciouvistas gorydetail it doesnt function well with windows core stuff like the font will get squished but in my prototypes i use it and it makes reading my games much easier', 'blakegc windows xp', 'the f7 has been cleared and is open for traffic in both directions motorists are still advised to keep windows rolled up while driving the pass however', 'maggieelmasry h will look in al

In [95]:
Doc_limpo = pd.DataFrame()
Doc_limpo['Treinamento']=limp
Doc_limpo['Relevancia']=Doc.Relevancia
Doc_limpo.head()

,Treinamento,Relevancia
0,evelynchartres oh thats good to know i have wi...,1
1,how to resolve a usb security key is not worki...,1
2,kumo ind llc korea aluminum amp upvc windows ...,0
3,full time heritage windows 2 1 braishfield fo...,0
4,sagaciouszu aciouvistas gorydetail it doesnt f...,1


In [96]:
def is_emoji(s):
    return s in UNICODE_EMOJI

def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

limp_final=[add_space(text) for text in limp]

print(limp_final)

['evelynchartres oh thats good to know i have windows 6 haha i really need a new computer', 'how to resolve a usb security key is not working on windows 10  https//tco/haugaqmie0 https//tco/gzja1nk7ug', 'kumo ind llc korea aluminum  amp upvc windows doors modern kitchen trading  showroom sheikh mohammed bin salem road ras al khaimah uae from saturday  thursday 0900am100pm then 400pm1000pm  factory umm al quwain uae from saturday  thursday 700am  700pm https//tco/kplxyscls4', 'full time heritage windows 2  1 braishfield football club herbra pitchero https//tco/zl7ulrkfe8 https//tco/ffqw0sgspv', 'sagaciouszu aciouvistas gorydetail it doesnt function well with windows core stuff like the font will get squished but in my prototypes i use it and it makes reading my games much easier', 'blakegc windows xp', 'the f7 has been cleared and is open for traffic in both directions motorists are still advised to keep windows rolled up while driving the pass however', 'maggieelmasry h will look in al

In [97]:
Doc_limpo = pd.DataFrame()
Doc_limpo['Treinamento']=limp_final
Doc_limpo['Relevancia']=Doc.Relevancia

In [98]:
Doc_limpo.Treinamento=Doc_limpo.Treinamento.str.replace('  ', ' ')
Doc_limpo.head()

,Treinamento,Relevancia
0,evelynchartres oh thats good to know i have wi...,1
1,how to resolve a usb security key is not worki...,1
2,kumo ind llc korea aluminum amp upvc windows d...,0
3,full time heritage windows 2 1 braishfield foo...,0
4,sagaciouszu aciouvistas gorydetail it doesnt f...,1


In [99]:
Doc_0=Doc_limpo[(Doc_limpo.Relevancia==0)]
Doc_1=Doc_limpo[(Doc_limpo.Relevancia==1)]

In [237]:
string0=" ".join(str(x) for x in Doc_0.Treinamento)
serie_0 = pd.Series(string0.split())
tabela_0=serie_0.value_counts(True).to_frame('Frequency')
tabela_0['Palavra']=tabela_0.index
tabela_0.head()

,Frequency,Palavra
the,0.037128,the
windows,0.035662,windows
to,0.020762,to
a,0.019541,a
and,0.019052,and


In [212]:
string1=" ".join(str(x) for x in Doc_1.Treinamento)
serie_1 = pd.Series(string1.split())
tabela_1=serie_1.value_counts(True).to_frame('Frequency')
tabela_1['Palavra']=tabela_1.index
tabela_1.head()

,Frequency,Palavra
windows,0.045016,windows
to,0.033031,to
a,0.025431,a
the,0.022508,the
and,0.020170,and


In [213]:
Doc = pd.read_excel('Windows.xlsx', sheet_name = 1)
Doc.head()

,Teste,Relevancia
0,bluekeep is a remote windows kernel use-after-...,1
1,"kumo ind. llc korea\naluminum &amp; u-pvc, win...",0
2,"anybody can spot me ""mcsa guide to identity wi...",1
3,thinking about moving to a @windows 10 device?...,1
4,@wocharlog windows wale raped,0


In [214]:
limp=[]
for i in Doc.Teste:
    a=re.sub('@','',i)
    a=re.sub(':','',a)
    a=re.sub("'",'',a)
    a=re.sub('-','',a)
    a=re.sub('#','',a)
    a=re.sub('"','',a)
    a=re.sub('!','',a)
    a=re.sub(';','',a)
    a=re.sub('\n',' ',a)
    a=re.sub('\t',' ',a)
    a=re.sub('&',' ',a)
    a=a.strip('()')
    a=a.replace('.','')
    a=a.replace('?','')
    a=a.replace(',','')
    limp.append(a)

In [215]:
Doc_limpo = pd.DataFrame()
Doc_limpo['Teste']=limp
Doc_limpo['Relevancia']=Doc.Relevancia
Doc_limpo.head()

,Teste,Relevancia
0,bluekeep is a remote windows kernel useafterfr...,1
1,kumo ind llc korea aluminum amp upvc windows ...,0
2,anybody can spot me mcsa guide to identity wit...,1
3,thinking about moving to a windows 10 device g...,1
4,wocharlog windows wale raped,0


In [288]:
def is_emoji(s):
    return s in UNICODE_EMOJI

def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

limp_final=[add_space(text) for text in limp]

print(limp_final)

['bluekeep is a remote windows kernel useafterfree vulnerability that could be used to create wormable attacks similar to the wannacry ransomware incident of may 2017  https//tco/diruolwurh', 'kumo ind llc korea aluminum  amp upvc windows doors modern kitchen trading  showroom sheikh mohammed bin salem road ras al khaimah uae from saturday  thursday 0900am100pm then 400pm1000pm  factory umm al quwain uae from saturday  thursday 700am  700pm https//tco/enbtewihrc', 'anybody can spot me mcsa guide to identity with windows server 2016 exam 70742 1st edition cengage 2018 isbn13 9781337400893 in pdf', 'thinking about moving to a windows 10 device gomodern to unlock cost savings and thirdparty security solutions check out the full forrester study for more https//tco/qsmgkebfvu', 'wocharlog windows wale raped', 'miguelandholly i’m riding with the windows down and that $1500 bid got me hollering so loud i frightened like 4 drivers around me  😆   bless up all these bidders and the miguelandholl

In [217]:
Doc_limpo = pd.DataFrame()
Doc_limpo['Teste']=limp_final
Doc_limpo['Relevancia']=Doc.Relevancia

Doc_limpo.Teste=Doc_limpo.Teste.str.replace('  ', ' ')
Doc_limpo.tail()

,Teste,Relevancia
195,hp probook 430 g2 intel core i55300u 4gb ram 5...,1
196,nothing beats a good case study 😎 here at fakr...,0
197,microsoft surface go mhn00015 2018 10inch lapt...,1
198,soledadobrien because “proper documentation” i...,0
199,man i have done fking everything i could think...,1


In [402]:
total = len(Doc_limpo[(Doc_limpo.Relevancia==0)]) + len(Doc_limpo[(Doc_limpo.Relevancia==1)])
P0= len(Doc_limpo[(Doc_limpo.Relevancia==0)])/total
P1= len(Doc_limpo[(Doc_limpo.Relevancia==1)])/total

In [403]:
lista_tot=[]
for a in Doc_limpo.Teste:
    lista_pal=a.split(' ')
    lista_tot.append(lista_pal)

lista_zero=[]
lista_um=[]
lista_comp=[]
for a in lista_tot:
    zero=1
    um=1
    for palavra in a:
        if palavra in tabela_0.Palavra:
            zero*=tabela_0.Frequency[palavra]
        if palavra in tabela_1.Palavra:
            um*=tabela_1.Frequency[palavra]
    lista_zero.append(zero*P0)
    lista_um.append(um*P1)

Doc_limpo['Prob_zero']=lista_zero
Doc_limpo['Prob_um']=lista_um

i=0
while i<len(lista_zero):
    if lista_zero[i]>lista_um[i]:
        lista_comp.append(1)
    else:
        lista_comp.append(0)
    i+=1


Doc_limpo['Relevancia_Comp']=lista_comp

In [404]:
Doc_limpo.head(20)

,Teste,Relevancia,Relevancia_Comp,Prob_zero,Prob_um
0,bluekeep is a remote windows kernel useafterfr...,1,1,6.554117e-35,9.753172e-36
1,kumo ind llc korea aluminum amp upvc windows d...,0,0,4.438562e-104,2.275782e-17
2,anybody can spot me mcsa guide to identity wit...,1,1,7.984149e-17,8.992448e-24
3,thinking about moving to a windows 10 device g...,1,1,6.120211e-41,2.139117e-54
4,wocharlog windows wale raped,0,0,1.890083e-02,2.115756e-02
5,miguelandholly i’m riding with the windows dow...,0,1,1.118028e-58,3.667621e-60
6,ridertd hotter than hell on bus 9377 and the w...,0,0,7.411665e-47,3.301016e-40
7,2019 | the original uservoice request for a li...,1,1,3.024062e-58,9.005450e-84
8,grantob frisbeedick macs and windows computers...,1,1,3.509505e-35,4.529916e-53
9,time to install the august windows10 patches —...,0,1,1.736482e-49,4.056529e-58


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**